# Lab 9

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-312/blob/main/book/labs/lab_09.ipynb)

## Overview

This lab introduces Google Earth Engine (GEE) for accessing and analyzing geospatial data in Python. You will explore diverse data types, including DEM, satellite imagery, and land cover datasets. You’ll gain experience creating cloud-free composites, visualizing temporal changes, and generating animations for time-series data.


## Objectives

By completing this lab, you will be able to:

1. Access and visualize Digital Elevation Model (DEM) data for a specific region.
2. Generate cloud-free composites from Sentinel-2 or Landsat imagery for a specified year.
3. Visualize National Agricultural Imagery Program (NAIP) data for U.S. counties.
4. Display watershed boundaries using Earth Engine styling.
5. Visualize land cover changes over time using split-panel maps.
6. Create a time-lapse animation for land cover changes over time in a region of your choice.


In [1]:
import ee
import geemap
from geemap.datasets import DATA

In [2]:
geemap.ee_initialize()


Successfully saved authorization token.


## Exercise 1: Visualizing DEM Data

Find a DEM dataset in the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets) and clip it to a specific area (e.g., your country, state, or city). Display it with an appropriate color palette. For example, the sample map below shows the DEM of the state of Colorado.

![](https://i.imgur.com/OLeSt7n.png)

In [4]:
m = geemap.Map(center=[40.906247614146224, -74.42645072937013], zoom=13)
dataset = ee.Image(DATA.USGS_3DEP_10m)
vis_params = {
    "min": 0,
    "max": 275,
    "opacity": .85,
    "palette": "terrain",
}
m.addLayer(dataset,vis_params, 'DEM')
m

Map(center=[40.906247614146224, -74.42645072937013], controls=(WidgetControl(options=['position', 'transparent…

In [5]:
if m.user_roi is not None:
    image = dataset.clip(m.user_roi)
    m.layers[1].visible = False
    m.add_layer(image, vis_params, 'Clipped DEM')

## Exercise 2: Cloud-Free Composite with Sentinel-2 or Landsat

Use Sentinel-2 or Landsat-9 data to create a cloud-free composite for a specific year in a region of your choice.

Use [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED) or [Landsat-9 data](https://developers.google.com/earth-engine/datasets/catalog/landsat-9) data to create a cloud-free composite for a specific year in a region of your choice. Display the imagery on the map with a proper band combination. For example, the sample map below shows a cloud-free false-color composite of Sentinel-2 imagery of the year 2021 for the state of Colorado. 

![](https://i.imgur.com/xkxpkS1.png)

In [6]:
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

In [7]:
geometry = ee.Geometry.Point([-112.07221984863283,36.10237644873644 ]
)
images = collection.filterBounds(geometry)

images = (
    collection.filterBounds(geometry)
    .filterDate("2022-01-01", "2022-12-31")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
)


In [8]:
m = geemap.Map()
image = images.median()

vis = {
    "min": 0.0,
    "max": 3000,
    "bands": ["B8", "B4", "B3"],
}

m.add_layer(image, vis, "Sentinel-2")
m.centerObject(geometry, 8)
m

Map(center=[36.10237644873645, -112.07221984863284], controls=(WidgetControl(options=['position', 'transparent…

## Exercise 3: Visualizing NAIP Imagery

Use [NAIP](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ) imagery to create a cloud-free imagery for a U.S. county of your choice. For example, the sample map below shows a cloud-free true-color composite of NAIP imagery for Knox County, Tennessee. Keep in mind that there might be some counties with the same name in different states, so make sure to select the correct county for the selected state.

![](https://i.imgur.com/iZSGqGS.png)

In [43]:
collection = ee.ImageCollection("USDA/NAIP/DOQQ")

counties =  ee.FeatureCollection("TIGER/2018/Counties")

In [45]:
county = counties.filter(
    ee.Filter.And(
        ee.Filter.eq("STATEFP", "34"),  
        ee.Filter.eq("NAME", "Morris") 
    )
)

ee_geometry = county.geometry()

naip = collection.filterBounds(ee_geometry).filterDate("2022-01-01", "2022-12-31")
naip_img = naip.median().clip(ee_geometry)


In [49]:
m = geemap.Map(center = [40.82731951134558, -74.44679260253908], zoom = 10)

vis = {
    "min": 0.0,
    "max": 255,
    "bands": ["R", "G", "B"],
}

m.addLayer(naip_img, vis, "NAIP")
m.addLayer(ee_geometry, {}, "Morris County")

m

Map(center=[40.82731951134558, -74.44679260253908], controls=(WidgetControl(options=['position', 'transparent_…

## Exercise 4: Visualizing Watershed Boundaries

Visualize the [USGS Watershed Boundary Dataset](https://developers.google.com/earth-engine/datasets/catalog/USGS_WBD_2017_HUC04) with outline color only, no fill color.

![](https://i.imgur.com/PLlNFq3.png)

In [58]:
usgs_wbd = ee.FeatureCollection("USGS/WBD/2017/HUC04")

style = {
    'color': 'blue',
    'width': 1,
    "fillColor": "00000000",
   
      }




In [59]:
m = geemap.Map(center = [39.232253141714914, -98.30566406250001], zoom = 4)

m.addLayer(usgs_wbd.style(**style), {}, "USGS WBD - Outline Only")

m

Map(center=[39.232253141714914, -98.30566406250001], controls=(WidgetControl(options=['position', 'transparent…

## Exercise 5: Visualizing Land Cover Change

Use the [USGS National Land Cover Database](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD) and [US Census States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States) to create a split-panel map for visualizing land cover change (2001-2019) for a US state of your choice. Make sure you add the NLCD legend to the map.

![](https://i.imgur.com/Au7Q5Ln.png)

In [80]:
m = geemap.Map(center = [40.19146303804063, -74.70428466796876], zoom = 8)

census = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'New Jersey'))
ee_geometry = census.geometry()

usgs_2001 = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD").select("landcover").filter(ee.Filter.eq('system:index', '2001')).first().clip(ee_geometry)
usgs_2019 = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD").select("landcover").filter(ee.Filter.eq('system:index', '2019')).first().clip(ee_geometry)


left_layer = geemap.ee_tile_layer(usgs_2001, {}, "USGS National Land Cover 2001")
right_layer = geemap.ee_tile_layer(usgs_2019, {}, "USGS National Land Cover 2019")

m.split_map(left_layer, right_layer)

m.add_legend(
    title="NLCD Land Cover Classification", builtin_legend="NLCD", height="455px"
)


m

Map(center=[40.19146303804063, -74.70428466796876], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Exercise 6: Creating a Landsat Timelapse Animation

Generate a timelapse animation using Landsat data to show changes over time for a selected region.

![Spain](https://github.com/user-attachments/assets/f12839c0-1c30-404d-b0ab-0fa12ce12d24)

In [99]:
m = geemap.Map()
roi = ee.Geometry.BBox( -74.035149, 40.39415, -73.962021, 40.491871)
m.add_layer(roi)
m.center_object(roi)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [100]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif="sandy_hook.gif",
    start_year=1984,
    end_year=2023,
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=5,
    title="Sandy Hook NJ",
    font_color="blue",
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: /Users/shawnclark/Desktop/geog-312/geog-312/book/labs/sandy_hook.gif


Output()